In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [35]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data/train",target_size=(300,286))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="data/validation", target_size=(300,286))

Found 2750 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [36]:
model = Sequential()

In [37]:
model.add(Conv2D(input_shape=(300,286,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))

In [38]:
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

In [39]:
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [40]:
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

In [41]:
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

In [42]:
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [43]:
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

In [44]:
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

In [45]:
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

In [46]:
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [47]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [48]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [49]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [50]:
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [51]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [52]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [53]:
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

In [54]:
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [55]:
model.add(Flatten())

In [56]:
model.add(Dense(units=4096,activation="relu"))

In [57]:
model.add(Dense(units=4096,activation="relu"))

In [58]:
model.add(Dense(units=5, activation="softmax"))

In [59]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 300, 286, 64)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 300, 286, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 150, 143, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 150, 143, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 150, 143, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 71, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 75, 71, 256)       295168    
__________

In [61]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_hand.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100
100/100 [==============================] - 9015s 90s/step - loss: 12.8564 - acc: 0.1948 - val_loss: 12.8842 - val_acc: 0.2006

Epoch 00001: val_acc improved from -inf to 0.20064, saving model to vgg16_hand.h5
Epoch 2/100
100/100 [==============================] - 8874s 89s/step - loss: 12.9079 - acc: 0.1992 - val_loss: 12.8842 - val_acc: 0.2006

Epoch 00002: val_acc did not improve from 0.20064
Epoch 3/100
100/100 [==============================] - 8810s 88s/step - loss: 12.7471 - acc: 0.2091 - val_loss: 12.8842 - val_acc: 0.2006

Epoch 00003: val_acc did not improve from 0.20064
Epoch 4/100
 99/100 [============================>.] - ETA: 1:24 - loss: 12.9572 - acc: 0.1961

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
from keras.preprocessing import image
img = image.load_img("test.jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("vgg16_hand.h5")
output = saved_model.predict(img)

In [ ]:
ans = output[0]

# compare index of max of ans

if output[0][0] > output[0][1]:
    print("cat")
else:
    print('dog')